In [1]:
# 필요한 라이브러리 설치
# !pip install pdfplumber anthropic nltk

import os
import pdfplumber
import anthropic
from IPython.display import display, Markdown
import nltk
from nltk.tokenize import sent_tokenize
from collections import defaultdict
nltk.download('punkt_tab')

# Anthropic API 키 설정
os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"  # 본인의 API 키로 교체하세요
client = anthropic.Client()

def identify_headings(page):
    """페이지에서 헤딩을 식별하는 함수"""
    chars = page.chars
    if not chars:
        return []
    
    # 기본 텍스트 특성 파악
    sizes = [char['size'] for char in chars if char['size'] is not None]
    if not sizes:
        return []
    
    avg_size = sum(sizes) / len(sizes)
    most_common_size = max(set(sizes), key=sizes.count)
    
    # 헤딩 레벨 기준 설정
    heading_levels = {
        'h1': avg_size * 1.5,
        'h2': avg_size * 1.1,
        'h3': avg_size * 1.05,
        'h4': avg_size * 1.00,
        'h5': avg_size * 0.95
    }
    
    # 라인별로 텍스트 그룹화
    lines = defaultdict(list)
    for char in chars:
        if char['size'] is not None:
            lines[round(char['top'])].append(char)
    
    headings = []
    for y, line_chars in lines.items():
        if not line_chars:
            continue
        
        # 라인의 특성 분석
        line_text = ''.join(char['text'] for char in sorted(line_chars, key=lambda x: x['x0']))
        line_size = sum(char['size'] for char in line_chars) / len(line_chars)
        is_bold = any('Bold' in (char.get('fontname', '') or '') for char in line_chars)
        is_caps = line_text.isupper()
        
        # 헤딩 레벨 결정
        heading_level = None
        if line_size >= heading_levels['h1'] or (line_size >= heading_levels['h2'] and is_bold and is_caps):
            heading_level = 'h1'
        elif line_size >= heading_levels['h2'] or (line_size >= heading_levels['h3'] and is_bold):
            heading_level = 'h2'
        elif line_size >= heading_levels['h3'] or (line_size >= heading_levels['h4'] and is_bold):
            heading_level = 'h3'
        elif line_size >= heading_levels['h4'] and (is_bold or is_caps):
            heading_level = 'h4'
        elif line_size >= heading_levels['h5'] and (is_bold or is_caps):
            heading_level = 'h5'
            
        if heading_level and len(line_text.strip()) > 0:
            headings.append({
                'text': line_text.strip(),
                'level': heading_level,
                'size': line_size,
                'font_properties': {
                    'is_bold': is_bold,
                    'is_caps': is_caps,
                },
                'position': {
                    'top': y,
                    'left': min(char['x0'] for char in line_chars),
                    'right': max(char['x1'] for char in line_chars)
                }
            })
    
    return sorted(headings, key=lambda x: x['position']['top'])

def read_pdf_with_headings(file_path):
    """PDF 파일을 읽어서 헤딩과 텍스트를 구조화하여 추출하는 함수"""
    try:
        with pdfplumber.open(file_path) as pdf:
            document_structure = []
            
            for page_num, page in enumerate(pdf.pages, 1):
                # 헤딩 식별
                headings = identify_headings(page)
                
                # 페이지의 전체 텍스트 추출
                text = page.extract_text()
                
                # 페이지 정보를 구조화
                page_structure = {
                    'page_number': page_num,
                    'headings': headings,
                    'text': text
                }
                
                document_structure.append(page_structure)
            
            # 구조화된 텍스트 생성
            formatted_text = ""
            for page in document_structure:
                # formatted_text += f"\n=== Page {page['page_number']} ===\n"
                
                # 헤딩 정보 추가
                if page['headings']:
                    # formatted_text += "\n[Headings]\n"
                    for heading in page['headings']:
                        level_marks = '#' * int(heading['level'][1])
                        # properties = f"[{heading['level']}, {'bold' if heading['font_properties']['is_bold'] else 'normal'}, {'CAPS' if heading['font_properties']['is_caps'] else 'normal'}]"
                        formatted_text += f"\n{level_marks} {heading['text']}\n\n"
                
                # 본문 텍스트 추가
                # formatted_text += "\n[Content]\n"
                formatted_text += page['text'] + "\n"
            
            return formatted_text, document_structure
            
    except Exception as e:
        print(f"PDF 읽기 오류: {str(e)}")
        return None, None

def split_text_into_chunks(text, chunk_size=2000):
    """텍스트를 문장 단위로 나누고 적절한 크기의 청크로 결합"""
    # 기존 함수와 동일

def translate_chunk(chunk, target_lang="한국어"):
    """개별 청크를 번역하는 함수"""
    # 기존 함수와 동일

def translate_with_claude(text, target_lang="한국어"):
    """전체 텍스트를 청크 단위로 나누어 번역하는 함수"""
    # 기존 함수와 동일

def translate_pdf(pdf_path, target_lang="한국어"):
    """PDF 파일을 읽고 번역하는 메인 함수"""
    # PDF 읽기
    text, structure = read_pdf_with_headings(pdf_path)
    if text is None:
        return
    
    # print("원본 텍스트 (헤딩 구조 포함):")
    # display(Markdown(f"```\n{text}\n```"))

    output_dir = "extracted"
    os.makedirs(output_dir, exist_ok=True)
    file_name = os.path.basename(pdf_path).rsplit(".", 1)[0]
    output_file = os.path.join(output_dir, file_name + f"_extracted.md")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)
    
    # # 텍스트 번역
    # print("\n번역 중...")
    # translated_text = translate_with_claude(text, target_lang)
    
    # if translated_text:
    #     print("\n번역 결과:")
    #     display(Markdown(f"```\n{translated_text}\n```"))
        
    #     # 결과를 파일로 저장
    #     output_file = pdf_path.rsplit('.', 1)[0] + f'_translated_{target_lang}.txt'
    #     with open(output_file, 'w', encoding='utf-8') as f:
    #         f.write(translated_text)
    #     print(f"\n번역 결과가 {output_file}에 저장되었습니다.")

[nltk_data] Downloading package punkt_tab to /Users/skcho/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# 사용 예시
pdf_path = "docs/Jack_Martin.pdf"  # 번역할 PDF 파일 경로
translate_pdf(pdf_path, target_lang="한국어")

In [2]:
# 사용 예시
pdf_path = "Neal/Neal-Ch2.pdf"  # 번역할 PDF 파일 경로
translate_pdf(pdf_path, target_lang="한국어")